# Keras

## Installation
I'm [installing Keras](http://www.pyimagesearch.com/2016/07/18/installing-keras-for-deep-learning/) with the Theano backend. I haven't bothered with GPU acceleration since I'm just using it on my own machine. I may go through the hassle on one of the servers I use. 

```
pip3 install Theano keras pillow
```

Then change the file `~/.keras/keras.jso` from:

```
{
    "floatx": "float32",
    "epsilon": 1e-07,
    "backend": "tensorflow",
    "image_dim_ordering": "tf"
}
```

To this:

```
{
    "floatx": "float32",
    "epsilon": 1e-07,
    "backend": "theano",
    "image_dim_ordering": "th"
}
```

## Background
- Keras is a deep learning library built on top of TensorFlow or Theano
- Focus of keras is the model
- Main type: sequeunce = linear stack of layers
- Create sequence by adding layers
- Then compile and fit
- Finally you can use the model to make predictions

## Pima Indians onset of diabetes dataset Tutorial
- Health data from Pima Indians
- Whether they developed diabetes within 5 years
- Binary classification problem: 0 (did not become diabetic) or 1 (became diabetic)

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

Using Theano backend.


### Load Dataset

In [4]:
# load pima indians dataset
dataset = numpy.loadtxt("../data/pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
X

array([[   6.   ,  148.   ,   72.   , ...,   33.6  ,    0.627,   50.   ],
       [   1.   ,   85.   ,   66.   , ...,   26.6  ,    0.351,   31.   ],
       [   8.   ,  183.   ,   64.   , ...,   23.3  ,    0.672,   32.   ],
       ..., 
       [   5.   ,  121.   ,   72.   , ...,   26.2  ,    0.245,   30.   ],
       [   1.   ,  126.   ,   60.   , ...,   30.1  ,    0.349,   47.   ],
       [   1.   ,   93.   ,   70.   , ...,   30.4  ,    0.315,   23.   ]])

### Keras Model
- Create sequence model
- Add layers to it

#### Input layer
- Needs to have the correct number of inputs
- Set when creating the first layer with the 'input_dim' argument
- Here we set it to 8 for the 8 input variables

#### How many layers and their types?
- Difficult to answer
- Requires trial and error
- Need network large enough to capture the structure of the problem
- This tutorial creates a fully-connected network structure with three layers.

#### Fully connected layers
- Defined using the dense class
- First argument: number of neurons in layer
- Second argument: Initialization method as init
- Third argument: activation function as activation
    
In this case, we initialize the network weights to a small random number generated from a uniform distribution (‘uniform‘), in this case between 0 and 0.05 because that is the default uniform weight initialization in Keras. Another traditional alternative would be ‘normal’ for small random numbers generated from a Gaussian distribution.

We will use the rectifier (‘relu‘) activation function on the first two layers and the sigmoid function in the output layer. It used to be the case that sigmoid and tanh activation functions were preferred for all layers. These days, better performance is achieved using the rectifier activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer. The first layer has 12 neurons and expects 8 input variables. The second hidden layer has 8 neurons and finally the output layer has 1 neuron to predict the class (onset of diabetes or not). The problem has 8 input variables and the first hidden layer has 12 neurons. Inputs are the columns of data, these are fixed. The Hidden layers in general are whatever we design based on whatever capacity we think we need to represent the complexity of the problem. In this case, we have chosen 12 neurons for the first hidden layer.

### Create Model

In [6]:
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

### Compile Model

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to make predictions for this problem.

We must specify the loss function to use to evaluate a set of weights, the optimizer used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as “binary_crossentropy“. We will also use the efficient gradient descent algorithm “adam” for no other reason that it is an efficient default.

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit Model
- Train/fit data on loaded data using the 'fit' method
- Epochs: Number of iterations through dataset - 'nb_epoch'
- Batch size: Number of instances evaluated before a weight update in the network is performed - 'batch_size'

For this problem, we will run for a small number of iterations (150) and use a relatively small batch size of 10. Again, these can be chosen experimentally by trial and error.

In [8]:
model.fit(X, Y, nb_epoch=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 0s - loss: 0.6826 - acc: 0.6328     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.6590 - acc: 0.6510     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.6475 - acc: 0.6549     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.6416 - acc: 0.6615     
Epoch 5/150
768/768 [==============================] - 0s - loss: 0.6216 - acc: 0.6745     
Epoch 6/150
768/768 [==============================] - 0s - loss: 0.6128 - acc: 0.6680     
Epoch 7/150
768/768 [==============================] - 0s - loss: 0.6018 - acc: 0.6927     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.5962 - acc: 0.6927     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.5991 - acc: 0.6953     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.5920 - acc: 0.6927     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.5905 - acc

### Evaluate Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the evaluate() function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

In [12]:
scores = model.evaluate(X, Y)
print(f'{model.metrics_names[1]} {scores[1]*100}')

 32/768 [>.............................] - ETA: 0sacc 78.90625


### Make Predictions
Making predictions is as easy as calling model.predict(). We are using a sigmoid activation function on the output layer, so the predictions will be in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [17]:
predictions = model.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0,